In [16]:
from clean import clean

import pandas as pd
import numpy as np
import category_encoders as ce
import xgboost as xgb
from sklearn.decomposition import PCA, TruncatedSVD


In [18]:
#%%time
# test
#encoder = ce.BinaryEncoder()
#X_train = encoder.fit_transform(clean(pd.read_csv('train_features.csv').sample(5678))[0])
## pca = TruncatedSVD(n_components=X_train.shape[1]-50, tol=2**3)
## pca.fit(X_train)
## pca.explained_variance_

np.linalg.svd(X_train.values.astype(np.float64))

LinAlgError: SVD did not converge

In [7]:

def encode(encoder, trainpath = 'train_features.csv', testpath = 'test_features.csv'): 
    ''' pass a fresh encoder instance from ce library. '''

    pca = TruncatedSVD(n_components='mle', tol=2**4)
    
    
    df_test = clean(pd.read_csv(testpath))
    X_train_ = encoder.fit_transform(clean(pd.read_csv(trainpath))[0])
    X_test_ = encoder.fit_transform(df_test[0])
    
    pca.fit(X_train_)
    
    X_train = pca.transform(X_train_)
    X_test = pca.transform(X_test_)
    
    return {'train': X_train, 'test': X_test, 'TEST_IDs': df_test[1]}

#dfs = encode(ce.BinaryEncoder())

In [ ]:
dfs['train'].head()

In [ ]:
y_train = pd.read_csv('train_labels.csv')['charged_off']
# # # y_test = pd.read_csv('test_labels.csv')['charged_off']
# sample_submission = pd.read_csv('sample_submission.csv')

dtrain = xgb.DMatrix(dfs['train'].values, y_train.values)
dtest = xgb.DMatrix(dfs['test'].values)

# specify parameters via map
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)
preds_df = pd.DataFrame({'id': dfs['TEST_IDs'], 'charged_off': preds})

assert all([x==y for x,y in zip(pd.read_csv('test_features.csv').id, preds_df.id)])

In [ ]:
preds_df.head()